# Export csv file(behavior_ms) from txt file

## Import packages

In [22]:
import csv
import sys
import os
import re
import pandas as pd
minian_path = "."
sys.path.insert(0, minian_path)
from minian.utilities import (
    match_information,
    match_path
)

## File path

Introduction for the parameters:
- dpath: the minian folder path
- inputfilename: the name of the origin behavior file
- outputfilename: the file with frame number, the real time and the number of behavior events happen at that frame
- timestamp_path: the path of the timeStamps.csv file which is the output from the miniscope

The explaination of `match_information` and `match_path` function:
- Use regular expression to get the information from the dpath.(Based on the current folder structure)
- Both of them are in the utilitis.py. You can change it if the folder structure changes.

In [ ]:
dpath="/N/project/Cortical_Calcium_Image/Miniscope data/05.2023_Tenth_group/AA060_D8/2023_05_09/10_26_58/Miniscope_2"
inputfilename = "2023-05-09_10h19m_Subject YC_AA060.txt"
mouseID, day, session = match_information(dpath)
mouse_path, video_path = match_path(dpath)
outputfilename = "behavior_ms.csv"
timestamp_path=os.path.join(video_path, "timeStamps.csv")

## reformat

In [ ]:
timestamp_pd=pd.read_csv(timestamp_path)

In [ ]:
def reformat(data_array,col_name, time_pd):
    '''
    This is the fuction to convert the X array, Y array, Z array to the pandas dataframe format. From list to dataframe columns.
    '''
    output_pd = time_pd
    # add a column called col_name and each value is 0
    output_pd.insert(len(output_pd.columns), col_name, 0)
    i = 0

    while (i < len(data_array)):
        # the unit of timStamps is ms but the unit of the txt file is sec. Here we want to uniform the unit.
        timestamp = data_array[i] * 1000
        # 
        output_pd.loc[(output_pd['Time Stamp (ms)'].shift(0)<= timestamp) & (output_pd['Time Stamp (ms)'].shift(-1)>timestamp), col_name] += 1
        i += 1

    print("{}: {}".format(col_name,output_pd[col_name].sum()))
    return output_pd

In [26]:
alp_number: int
ialp_number: int
rnfs_number: int
# X,Y,Z array is the same with the protocol
X_array = []
Y_array = []
Z_array = []
ALP_Timeout_array = []
file_path = os.path.join(mouse_path, inputfilename)
behavior_timeStamp = list(csv.reader(open(file_path)))

alp_number = int(float(re.findall(r"\d+\.?\d*", behavior_timeStamp[13][0])[0]))
ialp_number = int(
    float(re.findall(r"\d+\.?\d*", behavior_timeStamp[21][0])[0]))
rnfs_number = int(
    float(re.findall(r"\d+\.?\d*", behavior_timeStamp[14][0])[0]))
# alp=re.findall(r"\d+\.?\d*", )
print("mouseID: {}, day: {}, session: {}".format(mouseID, day, session))
print("alp_number: {}".format(alp_number))
print("ialp_number: {}".format(ialp_number))
print("rnfs_number: {}".format(rnfs_number))

print(behavior_timeStamp[36][0])  # X: reinforcement
for i in range(37, 237):
    for j in range(1, 6):
        X_array.append(
            float(re.findall(r"\d+\.?\d*", behavior_timeStamp[i][0])[j]))
# print(float(re.findall(r"\d+\.?\d*", behavior_timeStamp[237][0])[1]))
X_array.append(float(re.findall(r"\d+\.?\d*", behavior_timeStamp[237][0])[1]))
X_array = list(filter(lambda x: x != 0, X_array))
X_array_unique = []
for i in X_array:
    if i not in X_array_unique:
        X_array_unique.append(i)

print(behavior_timeStamp[238][0])  # Y: active lever press
for i in range(239, 439):
    for j in range(1, 6):
        Y_array.append(
            float(re.findall(r"\d+\.?\d*", behavior_timeStamp[i][0])[j]))
Y_array.append(float(re.findall(r"\d+\.?\d*", behavior_timeStamp[439][0])[1]))
Y_array = list(filter(lambda x: x != 0, Y_array))
Y_array_unique = []
for i in Y_array:
    if i not in Y_array_unique:
        Y_array_unique.append(i)

print(behavior_timeStamp[440][0])  # Z: inactive lever press
for i in range(441, 641):
    for j in range(1, 6):
        Z_array.append(
            float(re.findall(r"\d+\.?\d*", behavior_timeStamp[i][0])[j]))
Z_array.append(float(re.findall(r"\d+\.?\d*", behavior_timeStamp[641][0])[1]))
Z_array = list(filter(lambda x: x != 0, Z_array))
Z_array_unique = []
for i in Z_array:
    if i not in Z_array_unique:
        Z_array_unique.append(i)

for i in Y_array_unique:
    if i not in X_array_unique:
        ALP_Timeout_array.append(i)
# Merge all the events together
res = reformat(X_array, "RNFS", timestamp_pd)
# print(res)
res = reformat(Y_array, "ALP", res)
# print(res)
res = reformat(Z_array, "IALP", res)
# print(res)
res = reformat(ALP_Timeout_array, "ALP_Timeout", res)

res.to_csv(os.path.join(mouse_path, mouseID+"_" +
           day+"_"+"behavior_ms.csv"), index=None)

mouseID: AA060, day: D8, session: None
alp_number: 32
ialp_number: 23
rnfs_number: 27
X:
Y:
Z:
RNFS: 27
ALP: 32
IALP: 23
ALP_Timeout: 4


# Check

This is a check part to check is the output is correct.

In [27]:
check_pd = pd.read_csv(os.path.join(mouse_path,mouseID+"_"+day+"_"+"behavior_ms.csv"))

In [28]:
print(check_pd.loc[(check_pd["RNFS"]>1) | (check_pd["ALP"]>1) | (check_pd["IALP"]>1)])

      Frame Number  Time Stamp (ms)  Buffer Index  RNFS  ALP  IALP  \
9990          9990           335398             0     1    2     0   

      ALP_Timeout  
9990            0  


In [29]:
print(check_pd.loc[(check_pd["RNFS"]>=1) | (check_pd["ALP"]>=1) | (check_pd["ALP_Timeout"]>=1),["Frame Number","RNFS","ALP","ALP_Timeout"]])

        Frame Number  RNFS  ALP  ALP_Timeout
3919            3919     1    1            0
6112            6112     1    1            0
9990            9990     1    2            0
20438          20438     1    1            0
20489          20489     0    1            1
21784          21784     1    1            0
22842          22842     1    1            0
34663          34663     1    1            0
36227          36227     1    1            0
37609          37609     1    1            0
39088          39088     1    1            0
40831          40831     1    1            0
45111          45111     1    1            0
46651          46651     1    1            0
53630          53630     1    1            0
54732          54732     1    1            0
63634          63634     1    1            0
75965          75965     1    1            0
79504          79504     1    1            0
85345          85345     1    1            0
87129          87129     1    1            0
88689     